In [8]:
pip install gymnasium


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from time import sleep
import numpy as np
from IPython.display import clear_output
import gymnasium as gym
from gymnasium.envs.registration import register
import torch 
from torch import nn
import torch.optim as optim

ModuleNotFoundError: No module named 'torch'

In [4]:
#Import MiniPacMan environment class definition
from MiniPacManGym import MiniPacManEnv

In [5]:
#Register MiniPacMan in your gymnasium environments
register(
    id="MiniPacMan-v0",
    entry_point=MiniPacManEnv,
    max_episode_steps=20
)

In [6]:
#Create a MiniPacMan gymnasium environment
env = gym.make("MiniPacMan-v0", render_mode="human", frozen_ghost=True)

In [11]:
class QNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(6 * 6, 128)
        self.activation = nn.ReLU()
        self.linear2 = nn.Linear(128, 4)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        return x

NameError: name 'nn' is not defined

In [ ]:
q_network = QNetwork()
optimizer = optim.Adam(q_network.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

In [ ]:
#set hyperparams -- feel free to play with these!
gamma=0.999
alpha=0.95
num_episodes=10000

#initialize epsilon, Q
epsilon=1
Q=np.zeros((6,6,4)) #First two coordinates encode state, last encodes action

for e in range(num_episodes):
  new_obs,info=env.reset()
  new_obs=torch.tensor(new_obs.flatten(), dtype=torch.float32)
  done=False
  truncated=False
  steps=0

  while not done and not truncated: #Loop for one episode
    obs=new_obs

    #choose action
    t=np.random.random()
    if t>epsilon:
      with torch.no_grad():
        action=torch.argmax(q_network(obs)).item() #exploitation
    else:
      action= env.action_space.sample() #exploration

    #take a step:
    new_obs,reward, done, truncated, info=env.step(action)
    new_obs = torch.tensor(new_obs.flatten(), dtype=torch.float32)
    steps+=1
    new_pos=np.argwhere(new_obs==1)[0] #next pacman position

    #compute target q-value
    with torch.no_grad():
      target_q = reward + gamma * torch.max(q_network(new_obs))

    #current q-value
    current_q = q_network(obs)[action]

    #compute loss
    loss = loss_fn(current_q, target_q)

    #Q-network update rule:
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  #reduce episilon if its not too low
  #Should be close to zero after 50 - 60% of episodes, and then level off
  epsilon=max(0.01, epsilon - 1.0/num_episodes)

  #periodic reporting:
  if e%100==0:
    print(f'episode: {e}, steps: {steps}, epsilon: {epsilon}, win: {reward==10}')



In [ ]:
#Run this code cell to see your trained agent in action!

obs, info = env.reset()
done = False
truncated = False

while not done and not truncated:
    env.render()
    obs = torch.tensor(obs.flatten(), dtype=torch.float32)
    with torch.no_grad():
        action = torch.argmax(q_network(obs)).item()
    obs, reward, done, truncated, info = env.step(action)
    sleep(1)
    clear_output(wait=True)

env.close()